In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
import os
import re

In [2]:
def get_trm():
    response = requests.get('https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@1/latest/currencies/usd/cop.json')

    if response.status_code == 200:
        data = response.json()
        return (data['cop']) # Imprime el valor de la tasa de cambio
    else:
        print("Ha ocurrido un error al hacer la solicitud.")
        return 5000
        

In [8]:
##Leer la pagina de busqueda para los 16 forros mas buscados ordenados por pedidos y envio gratis si supera la marca
browser = uc.Chrome()

url = 'https://www.amazon.com/s?k=ssd&rh=p_n_is_free_shipping%3A10236242011&s=exact-aware-popularity-rank&dc&language=es&__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2HNB4W2XNM2PC&qid=1681448085&rnid=10236241011&sprefix=ssd%2Caps%2C262&ref=sr_st_exact-aware-popularity-rank&ds=v1%3A3Tr9HRE8Yp%2BD%2BnwryWhRB8sFgsNLs9Gf0oAK6kTOXPk'
url2 = 'https://www.amazon.com/-/es/s?k=ssd&rh=p_n_is_free_shipping%3A10236242011&s=exact-aware-popularity-rank&dc&page=2&language=es&__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2HNB4W2XNM2PC&qid=1681448126&rnid=10236241011&sprefix=ssd%2Caps%2C262&ref=sr_pg_2'
browser.get(url)

browser.implicitly_wait(60)

html = browser.page_source

soup = bs(html,'lxml')

browser.get(url2)

browser.implicitly_wait(69)

html2 = browser.page_source

soup2 = bs(html,'lxml')

#Cerrar pestaña y navegador
#browser.close()
#browser.quit()

In [ ]:
#Crear array con las tarjetas de cada uno de los elementos (la tarjeta 0 es informacion)
cards = soup.find_all('div', class_="a-section a-spacing-small a-spacing-top-small")[1:]
cards2 = soup2.find_all('div', class_="a-section a-spacing-small a-spacing-top-small")[1:]
cards = cards + cards2
cards


In [9]:
def extract_discount(text):
    # Definir la expresión regular
    regex = r"Ahorra\s+(\d+)\s*%"

    # Buscar el número usando la expresión regular
    match = re.search(regex, text)

    # Devolver el número como un entero si se encontró una coincidencia
    if match:
        return int(match.group(1))

    # Devolver None si no se encontró una coincidencia
    return None


In [12]:
#Definir funciones para extraer info primero de la card y luego del link de la card

def get_name(card):
    name = card.find('span', class_='a-size-medium a-color-base a-text-normal').text
    return name
def get_price(card):
    price = soup.find('span',{'class':'a-price-whole'}).text + soup.find('span',{'class':'a-price-fraction'}).text
    return float(price)*get_trm()
def get_discount(card):
    discount_text = soup.find('span',{'class':'a-size-base s-highlighted-text-padding aok-inline-block s-coupon-highlight-color'}).text
    discount = extract_discount(discount_text)
    return discount
def get_link(card):
    link = card.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    href = 'https://www.amazon.com.co' + link.get('href')
    return href
   

## Ahora a extraer de cada pagina

In [13]:
ssd_test_link = get_link(cards[0])
ssd_test_link

'https://www.amazon.com.co/-/es/PCIe-NVMe-interno-juegos-MZ-V8P2T0B/dp/B08RK2SR23/ref=sr_1_1?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2HNB4W2XNM2PC&keywords=ssd&qid=1681448276&refinements=p_n_is_free_shipping%3A10236242011&rnid=10236241011&sprefix=ssd%2Caps%2C262&sr=8-1'

In [15]:
# Abrir la pagina web
PATH = '../usr/lib/chromium-browser/'
driver = webdriver.Chrome(PATH)
driver.get(ssd_test_link)
driver.maximize_window()

/tmp/ipykernel_20533/3142230861.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


WebDriverException: Message: unknown error: failed to change window state to 'normal', current state is 'maximized'
  (Session info: chrome=112.0.5615.49)
Stacktrace:
#0 0x55ca09918603 <unknown>
#1 0x55ca096698b6 <unknown>
#2 0x55ca096481ff <unknown>
#3 0x55ca09646650 <unknown>
#4 0x55ca09646208 <unknown>
#5 0x55ca096f1c06 <unknown>
#6 0x55ca096c6402 <unknown>
#7 0x55ca096de9a9 <unknown>
#8 0x55ca096c61d3 <unknown>
#9 0x55ca09698fb8 <unknown>
#10 0x55ca0969a0ee <unknown>
#11 0x55ca098df046 <unknown>
#12 0x55ca098e23c3 <unknown>
#13 0x55ca098e2c5a <unknown>
#14 0x55ca098eab0e <unknown>
#15 0x55ca098e33ec <unknown>
#16 0x55ca098b9ea8 <unknown>
#17 0x55ca09903318 <unknown>
#18 0x55ca099034c3 <unknown>
#19 0x55ca09910e34 <unknown>
#20 0x7f6db12e3609 <unknown>


In [20]:
size_data = driver.find_element(By.XPATH,'//*[@id="tp-inline-twister-dim-values-container"]/ul')
size_data

<selenium.webdriver.remote.webelement.WebElement (session="8816584e7f7471eadf28b5c7c9f510a0", element="658ad99f-5294-4ba7-9e83-b6cd8111c855")>